In [4]:
!pip install --user customtkinter


   ---------------------------------------- 0.0/296.1 kB ? eta -:--:--
   -- ------------------------------------ 20.5/296.1 kB 320.0 kB/s eta 0:00:01
   -- ------------------------------------ 20.5/296.1 kB 320.0 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/296.1 kB 363.1 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/296.1 kB 544.7 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/296.1 kB 479.3 kB/s eta 0:00:01
   -------------------------- ----------- 204.8/296.1 kB 689.9 kB/s eta 0:00:01
   ------------------------------ ------- 235.5/296.1 kB 758.5 kB/s eta 0:00:01
   ------------------------------------ - 286.7/296.1 kB 768.0 kB/s eta 0:00:01
   -------------------------------------- 296.1/296.1 kB 762.2 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tkinter as tk
from tkinter import filedialog
import pygame
import os
import customtkinter as ctk

# Initialize Pygame mixer
pygame.mixer.init()

# Music Player Class
class MusicPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Music Player")
        self.master.geometry("800x500")

        # Set default theme to dark
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("dark-blue")

        self.current_theme = "dark"
        self.set_colors()

        # Create frames
        self.sidebar_frame = ctk.CTkFrame(master, width=200, fg_color=self.top_bg_color, corner_radius=0)
        self.sidebar_frame.pack(fill=tk.Y, side=tk.LEFT, anchor=tk.NW)

        self.main_frame = ctk.CTkFrame(master, fg_color=self.bottom_bg_color, corner_radius=0)
        self.main_frame.pack(fill=tk.BOTH, expand=True, side=tk.RIGHT)

        # Sidebar for Playlist
        self.playlist_sidebar = tk.Listbox(self.sidebar_frame, bg=self.listbox_bg_color, fg=self.text_color, selectbackground=self.slider_color, font=("Helvetica", 12))
        self.playlist_sidebar.pack(pady=20, padx=10, fill=tk.BOTH, expand=True)

        # Playlist (display only song names)
        self.playlist_frame = ctk.CTkFrame(self.main_frame, fg_color=self.top_bg_color)
        self.playlist_frame.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)

        self.playlist = tk.Listbox(self.playlist_frame, bg=self.listbox_bg_color, fg=self.text_color, selectbackground=self.slider_color, font=("Helvetica", 12))
        self.playlist.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)

        # Controls Frame
        self.controls_frame = ctk.CTkFrame(self.main_frame, fg_color=self.bottom_bg_color)
        self.controls_frame.pack(pady=20)

        # Buttons
        self.play_button = ctk.CTkButton(self.controls_frame, text="Play", command=self.play_song, fg_color=self.button_color, text_color=self.text_color)
        self.pause_button = ctk.CTkButton(self.controls_frame, text="Pause", command=self.pause_song, fg_color=self.button_color, text_color=self.text_color)
        self.stop_button = ctk.CTkButton(self.controls_frame, text="Stop", command=self.stop_song, fg_color=self.button_color, text_color=self.text_color)
        self.next_button = ctk.CTkButton(self.controls_frame, text="Next", command=self.next_song, fg_color=self.button_color, text_color=self.text_color)

        # Grid Layout for Buttons
        self.play_button.grid(row=0, column=0, padx=10)
        self.pause_button.grid(row=0, column=1, padx=10)
        self.stop_button.grid(row=0, column=2, padx=10)
        self.next_button.grid(row=0, column=3, padx=10)

        # Skip Buttons
        self.skip_buttons_frame = ctk.CTkFrame(self.controls_frame, fg_color=self.bottom_bg_color)
        self.skip_buttons_frame.grid(row=1, column=0, columnspan=4, pady=10)

        self.skip_backward_button = ctk.CTkButton(self.skip_buttons_frame, text="-5s", command=self.skip_backward, fg_color=self.button_color, text_color=self.text_color)
        self.skip_forward_button = ctk.CTkButton(self.skip_buttons_frame, text="+5s", command=self.skip_forward, fg_color=self.button_color, text_color=self.text_color)

        # Skip Buttons Layout
        self.skip_backward_button.grid(row=0, column=0, padx=10)
        self.skip_forward_button.grid(row=0, column=1, padx=10)

        # Add Song Button
        self.add_song_button = ctk.CTkButton(self.controls_frame, text="Add Song", command=self.add_song, fg_color=self.button_color, text_color=self.text_color)
        self.add_song_button.grid(row=2, column=0, columnspan=4, pady=10)

        # Volume Control
        self.volume = tk.DoubleVar()
        self.volume.set(0.5)  # Set initial volume to 50%
        self.volume_slider = ctk.CTkSlider(self.controls_frame, from_=0, to=1, number_of_steps=100, command=self.set_volume, fg_color=self.slider_color, button_color=self.button_color)
        self.volume_slider.set(self.volume.get())
        self.volume_slider.grid(row=3, column=0, columnspan=4, pady=10, padx=20, sticky="ew")

        # Save and Load Playlist
        self.save_playlist_button = ctk.CTkButton(self.controls_frame, text="Save Playlist", command=self.save_playlist, fg_color=self.button_color, text_color=self.text_color)
        self.load_playlist_button = ctk.CTkButton(self.controls_frame, text="Load Playlist", command=self.load_playlist, fg_color=self.button_color, text_color=self.text_color)
        self.save_playlist_button.grid(row=4, column=0, columnspan=2, pady=5)
        self.load_playlist_button.grid(row=4, column=2, columnspan=2, pady=5)

        # Theme Switcher
        self.theme_switch_button = ctk.CTkButton(self.controls_frame, text="Switch to Light Theme", command=self.switch_theme, fg_color=self.button_color, text_color=self.text_color)
        self.theme_switch_button.grid(row=5, column=0, columnspan=4, pady=5)

        # Song Title Display with Typewriter Animation
        self.song_title_var = tk.StringVar()
        self.song_title_label = ctk.CTkLabel(self.controls_frame, textvariable=self.song_title_var, font=("Courier", 16))
        self.song_title_label.grid(row=6, column=0, columnspan=4, pady=5)

    def set_colors(self):
        if self.current_theme == "dark":
            self.top_bg_color = "#1c1c1c"
            self.bottom_bg_color = "#2c2c2c"
            self.listbox_bg_color = "#1c1c1c"
            self.button_color = "#444444"
            self.text_color = "white"
            self.slider_color = "#646464"
        else:
            self.top_bg_color = "#f5f5f5"
            self.bottom_bg_color = "#e0e0e0"
            self.listbox_bg_color = "white"
            self.button_color = "#cccccc"
            self.text_color = "black"
            self.slider_color = "#aaaaaa"

    def switch_theme(self):
        if self.current_theme == "dark":
            ctk.set_appearance_mode("light")
            ctk.set_default_color_theme("blue")
            self.current_theme = "light"
            self.theme_switch_button.configure(text="Switch to Dark Theme")
        else:
            ctk.set_appearance_mode("dark")
            ctk.set_default_color_theme("dark-blue")
            self.current_theme = "dark"
            self.theme_switch_button.configure(text="Switch to Light Theme")

        self.set_colors()
        self.update_colors()

    def update_colors(self):
        self.master.configure(bg=self.top_bg_color)
        self.sidebar_frame.configure(fg_color=self.top_bg_color)
        self.main_frame.configure(fg_color=self.bottom_bg_color)
        self.playlist_frame.configure(fg_color=self.top_bg_color)
        self.playlist.configure(bg=self.listbox_bg_color, fg=self.text_color, selectbackground=self.slider_color)
        self.playlist_sidebar.configure(bg=self.listbox_bg_color, fg=self.text_color, selectbackground=self.slider_color)
        self.controls_frame.configure(fg_color=self.bottom_bg_color)
        self.play_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.pause_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.stop_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.next_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.add_song_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.volume_slider.configure(fg_color=self.slider_color, button_color=self.button_color)
        self.save_playlist_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.load_playlist_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.theme_switch_button.configure(fg_color=self.button_color, text_color=self.text_color)
        self.song_title_label.configure(text_color=self.text_color)

    def add_song(self):
        song = filedialog.askopenfilename(initialdir="/", title="Choose A Song", filetypes=(("mp3 Files", "*.mp3"),))
        if song:
            song_name = os.path.basename(song)
            self.playlist.insert(tk.END, song_name)
            self.playlist_sidebar.insert(tk.END, song_name)
            self.playlist.song_paths = getattr(self.playlist, 'song_paths', [])
            self.playlist.song_paths.append(song)  # Save the full path for playback

    def play_song(self):
        song_name = self.playlist.get(tk.ACTIVE)
        if song_name:
            song = [path for path in getattr(self.playlist, 'song_paths', []) if os.path.basename(path) == song_name]
            if song:
                pygame.mixer.music.load(song[0])
                pygame.mixer.music.play(loops=0)
                self._typewriter(song_name)

    def _typewriter(self, text):
        self.song_title_var.set("")
        for i in range(len(text) + 1):
            self.master.after(i * 100, lambda i=i: self.song_title_var.set(text[:i]))

    def pause_song(self):
        pygame.mixer.music.pause()

    def stop_song(self):
        pygame.mixer.music.stop()
        self.song_title_var.set("")  # Clear the song title

    def skip_forward(self):
        if pygame.mixer.music.get_busy():
            current_pos = pygame.mixer.music.get_pos() / 1000  # Convert to seconds
            song_index = self.playlist.curselection()
            if song_index:
                song = [path for path in getattr(self.playlist, 'song_paths', []) if os.path.basename(path) == self.playlist.get(tk.ACTIVE)]
                if song:
                    song_length = pygame.mixer.Sound(song[0]).get_length()
                    new_pos = min(current_pos + 5, song_length)  # Skip forward 5 seconds
                    pygame.mixer.music.play(start=new_pos)

    def skip_backward(self):
        if pygame.mixer.music.get_busy():
            current_pos = pygame.mixer.music.get_pos() / 1000  # Convert to seconds
            new_pos = max(current_pos - 5, 0)  # Skip backward 5 seconds
            pygame.mixer.music.play(start=new_pos)

    def next_song(self):
        current_index = self.playlist.curselection()[0]
        next_index = (current_index + 1) % self.playlist.size()
        self.playlist.select_clear(0, tk.END)
        self.playlist.select_set(next_index)
        self.play_song()

    def set_volume(self, val):
        pygame.mixer.music.set_volume(float(val))

    def save_playlist(self):
        playlist = self.playlist.get(0, tk.END)
        with open('playlist.m3u', 'w') as f:
            f.writelines('\n'.join(self.playlist.song_paths))

    def load_playlist(self):
        with open('playlist.m3u', 'r') as f:
            songs = f.readlines()
        for song in songs:
            song = song.strip()
            self.playlist.insert(tk.END, os.path.basename(song))
            self.playlist_sidebar.insert(tk.END, os.path.basename(song))
        self.playlist.song_paths = songs

# Initialize Tkinter window
root = ctk.CTk()
app = MusicPlayer(root)
root.mainloop()
